<a href="https://colab.research.google.com/github/guard1000/SKT_AI_WG/blob/master/210409_%ED%95%99%EC%8A%B5%EC%8B%9C%EA%B0%84%ED%98%84%ED%96%89%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **사용자 - 파일명만 설정해 주세요**

In [ ]:
##################################################
########  업로드한 파일명을 알려주세요 ###########

file_name = 'data.xlsx'           #  파일명

##################################################
### 시트 명이 따로 변경되었을 경우 아래를 수정 ###
sheet_name_suni = '시트5_SUNI'
sheet_name_suni_개인 = '시트6_SUNI 개인학습'
sheet_name_집합과정 = '시트7_집합과정'
sheet_name_온라인 = '시트8_온라인'
sheet_name_studymate = '시트11_StudyMate'
sheet_name_TLP_사내집합 = '시트12_ TLP_사내 집합 과정'
sheet_name_TLP_IDP = '시트13_ TLP_IDP 등록 과정'
sheet_name_TLP_IDP_미등록 = '시트14_ TLP_IDP 미등록 과정'
sheet_name_TLP_IDP_T_earning = '시트15_ TLP_T-earning'


sheet_name_구성원정보 = '시트4_ 인력DB'

##################################################
# 생성될 output_종합 파일 이름을 변경하고 싶을 경우 수정 
output_종합_파일명 = 'output_종합.csv'

# 엑셀 파일 생성용 모듈 
!pip install XlsxWriter

     |████████████████████████████████| 153kB 5.6MB/s 


**아래 PART1과 PART2는 별도의 수정하지 말고 그냥 재생버튼(셀실행버튼) 만 클릭해 주세요.**


**PART1 실행시** -> output 파일이 생성된 것을 확인할 수 있습니다.

**PART2 실행시** -> 본 노트북 제일 하단의 search() 셀을 실행시 사번으로 검색을 할 수 있습니다.

# **PART 1. [종합]**

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

import time
time_start = time.time() # timer setting

cnt = 0

# 1-1. output scheme 정의
output_종합 = pd.DataFrame(columns= ['사번','성명','최상위 Lv','실/본부 Lv', '팀 Lv', '직책', '사내 메일',
                                   '과정명','학습시간(시간)','이수여부','채널','구분'])


# 1-2. load employee_data tables
구성원정보 = pd.read_excel(file_name, sheet_name = sheet_name_구성원정보)
구성원메일 = list(구성원정보['사내 메일'])     # e-mail lowercase
구성원메일[10].lower()
for _ in range(len(구성원메일)):
  구성원메일[_] = 구성원메일[_].lower()
구성원정보['사내 메일'] = 구성원메일


# 1-3. Collection - suni
time_suni = time.time() # timer setting

suni = pd.read_excel(file_name, sheet_name = sheet_name_suni)
suni = suni[suni['result.success'] == 'Passed']               # suni는 passed만 처리

for _ in range(len(suni)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0

  # suni table에서 뽑을 수 있는 정보
  메일, 성명 = suni.iloc[_]['actor.mbox'], suni.iloc[_]['actor.name']
  과정명, 학습시간, 이수여부 = suni.iloc[_]['object.extension.cube.name'], format(suni.iloc[_]['object.ack_time(분)']/60.0, ".3f"), suni.iloc[_]['result.success']
  채널, 구분 = 'mySUNI', 'mySUNI'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사내 메일'] == 메일] # suni는 이메일로 로그인 하므로, key값 가능
  try:
    사번, 최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['사번'], 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    사번=최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print(cnt, '번째 row 생성 완료 소요시간:', time.time() - time_suni)   

time_suni = time.time() - time_suni # suni processing time
print('[suni] 처리 완료\n 소요시간:', time_suni)
print('------------------------------------------')     


# 1-4. Collection - suni_개인
suni_개인 = pd.read_excel(file_name, sheet_name = sheet_name_suni_개인)
suni_개인 = suni_개인[suni_개인['상태'] == '승인완료']       # suni_개인은 승인완료만 처리

time_suni_개인 = time.time() # timer setting

for _ in range(len(suni_개인)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0
  
  # suni_개인 table에서 뽑을 수 있는 정보
  메일, 성명 = suni_개인.iloc[_]['생성자 eMail'], suni_개인.iloc[_]['생성자']
  과정명, 이수여부 = suni_개인.iloc[_]['교육명'],  suni_개인.iloc[_]['상태']
  학습시간 = (suni_개인.iloc[_]['교육시간']).split()
  학습시간 = format(float(학습시간[0])+float(학습시간[2])/60.0 , ".3f")
  채널, 구분 = 'SUNI 개인주도학습', 'mySUNI'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사내 메일'] == 메일] # suni는 이메일로 로그인 하므로, key값 가능
  try:
    사번, 최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['사번'], 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    사번=최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print('time_suni_개인', cnt, '번째 row 생성 완료 소요시간:', time.time() - time_suni_개인)   

time_suni_개인 = time.time() - time_suni_개인 # time_suni_개인 processing time
print('[suni_개인] 처리 완료\n 소요시간:', time_suni_개인)
print('------------------------------------------')            


# 1-5. Collection - TLP 집합
집합과정 = pd.read_excel(file_name, sheet_name = sheet_name_집합과정)
집합과정 = 집합과정[집합과정['이수상태'] == '이수']       # 집합과정은 이수만 처리
집합과정 = 집합과정[집합과정['과정명'] != '[Infra센터] 신입 직무교육']       # 최종 버전에선 이부분 코드 삭제! ##############################

time_집합과정 = time.time() # timer setting

for _ in range(len(집합과정)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0
  
  # 집합과정 table에서 뽑을 수 있는 정보
  사번, 메일, 성명 = 집합과정.iloc[_]['사번'], 집합과정.iloc[_]['이메일'], 집합과정.iloc[_]['이름']
  과정명, 이수여부 = 집합과정.iloc[_]['과정명'],  집합과정.iloc[_]['이수상태'] 
  학습시간 = format(float(집합과정.iloc[_]['교육시간']), ".3f")
  채널, 구분 = 'TLP 집합', 'TLP'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사번'] == 사번] # TLP는사번이 key
  try:
    최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print('집합과정', cnt, '번째 row 생성 완료 소요시간:', time.time() - time_집합과정)   

time_집합과정 = time.time() - time_집합과정 # time_집합과정 processing time
print('[TLP_집합과정] 처리 완료\n 소요시간:', time_집합과정)
print('------------------------------------------')         


# 1-6. Collection - TLP 온라인
온라인 = pd.read_excel(file_name, sheet_name = sheet_name_온라인)
온라인 = 온라인[온라인['이수상태'] == '이수']       # 온라인은 이수만 처리

time_온라인 = time.time() # timer setting

for _ in range(len(온라인)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0
  
  # 온라인 table에서 뽑을 수 있는 정보
  사번, 메일, 성명 = 온라인.iloc[_]['사번'], 온라인.iloc[_]['이메일'], 온라인.iloc[_]['이름']
  과정명, 이수여부 = 온라인.iloc[_]['과정명'],  온라인.iloc[_]['이수상태'] 
  학습시간 = format(float(온라인.iloc[_]['재생시간(분)'])/60.0, ".3f")
  채널, 구분 = 'TLP 온라인', 'TLP'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사번'] == 사번] # TLP는사번이 key
  try:
    최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print('온라인', cnt, '번째 row 생성 완료 소요시간:', time.time() - time_온라인)   

time_온라인 = time.time() - time_온라인 # time_온라인 processing time
print('[TLP_온라인] 처리 완료\n 소요시간:', time_온라인)
print('------------------------------------------')        


# 1-7. Collection - StudyMate
StudyMate = pd.read_excel(file_name, sheet_name = sheet_name_studymate)
#StudyMate = StudyMate[StudyMate['이수상태'] == '이수']       # StudyMate는 전부 다 처리?

time_StudyMate = time.time() # timer setting

for _ in range(len(StudyMate)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0
  
  # StudyMate table에서 뽑을 수 있는 정보
  사번, 메일, 성명 = StudyMate.iloc[_]['사번'], StudyMate.iloc[_]['이메일'], StudyMate.iloc[_]['이름']
  과정명, 이수여부 = StudyMate.iloc[_]['모임명'], 이수여부 # 이수여부 공란
  학습시간 = format(float(StudyMate.iloc[_]['학습시간 2배(분)'])/60.0, ".3f")
  채널, 구분 = 'TLP Study mate', 'TLP'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사번'] == 사번] # TLP는사번이 key
  try:
    최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print('StudyMate', cnt, '번째 row 생성 완료 소요시간:', time.time() - time_온라인)   

time_StudyMate = time.time() - time_StudyMate # time_StudyMate processing time
print('[TLP_StudyMate] 처리 완료\n 소요시간:', time_StudyMate)
print('------------------------------------------')       


# 1-8. Collection - TLP_사내 집합 과정
사내집합 = pd.read_excel(file_name, sheet_name = sheet_name_TLP_사내집합)
사내집합 = 사내집합[사내집합['이수여부'] == 'Y']       # TLP_사내집합 은 이수여부 'Y' 만

time_사내집합 = time.time() # timer setting

for _ in range(len(사내집합)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0
  
  # TLP 사내 집합 table에서 뽑을 수 있는 정보
  사번, 메일, 성명 = 사내집합.iloc[_]['사번'], 사내집합.iloc[_]['이메일'], 사내집합.iloc[_]['성명']
  과정명, 이수여부 = 사내집합.iloc[_]['과정명'], 사내집합.iloc[_]['이수여부']
  학습시간 = format(float(사내집합.iloc[_]['학습시간(분)'])/60.0, ".3f")
  채널, 구분 = '러닝월드 사내집합', 'TLP'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사번'] == 사번] # TLP는사번이 key
  try:
    최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print('사내집합', cnt, '번째 row 생성 완료 소요시간:', time.time() - time_사내집합)   

time_사내집합 = time.time() - time_사내집합 # time_사내집합 processing time
print('[TLP_사내 집합] 처리 완료\n 소요시간:', time_사내집합)
print('------------------------------------------')     


# 1-9. Collection - TLP_IDP
TLP_IDP = pd.read_excel(file_name, sheet_name = sheet_name_TLP_IDP)
TLP_IDP = TLP_IDP[TLP_IDP['이수여부'] == 'Y']       # TLP_IDP 은 이수여부 'Y' 만

time_TLP_IDP = time.time() # timer setting

for _ in range(len(TLP_IDP)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0
  
  # TLP_IDP table에서 뽑을 수 있는 정보
  사번, 메일, 성명 = TLP_IDP.iloc[_]['사번'], TLP_IDP.iloc[_]['이메일'], TLP_IDP.iloc[_]['성명']
  과정명, 이수여부 = TLP_IDP.iloc[_]['과정명'], TLP_IDP.iloc[_]['이수여부']
  학습시간 = format(float(TLP_IDP.iloc[_]['학습시간(분)'])/60.0, ".3f")
  채널, 구분 = '러닝월드 IDP', 'TLP'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사번'] == 사번] # TLP는사번이 key
  try:
    최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print('TLP_IDP', cnt, '번째 row 생성 완료 소요시간:', time.time() - time_TLP_IDP)   

time_TLP_IDP = time.time() - time_TLP_IDP # time_TLP_IDP processing time
print('[TLP_IDP] 처리 완료\n 소요시간:', time_TLP_IDP)
print('------------------------------------------')     


# 1-10. Collection - TLP_IDP_미등록
TLP_IDP_미등록 = pd.read_excel(file_name, sheet_name = sheet_name_TLP_IDP_미등록)
TLP_IDP_미등록 = TLP_IDP_미등록[TLP_IDP_미등록['이수여부'] == 'Y']       # TLP_IDP_미등록 은 이수여부 'Y' 만

time_TLP_IDP_미등록 = time.time() # timer setting

for _ in range(len(TLP_IDP_미등록)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0
  
  # TLP_IDP_미등록 table에서 뽑을 수 있는 정보
  사번, 메일, 성명 = TLP_IDP_미등록.iloc[_]['사번'], TLP_IDP_미등록.iloc[_]['이메일'], TLP_IDP_미등록.iloc[_]['성명']
  과정명, 이수여부 = TLP_IDP_미등록.iloc[_]['과정명'], TLP_IDP_미등록.iloc[_]['이수여부']
  학습시간 = format(float(TLP_IDP_미등록.iloc[_]['학습시간(분)'])/60.0, ".3f")
  채널, 구분 = '러닝월드 미등록IDP', 'TLP'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사번'] == 사번] # TLP는사번이 key
  try:
    최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print('TLP_IDP_미등록', cnt, '번째 row 생성 완료 소요시간:', time.time() - time_TLP_IDP_미등록)   

time_TLP_IDP_미등록 = time.time() - time_TLP_IDP_미등록 # time_TLP_IDP미등록 processing time
print('[TLP_IDP_미등록] 처리 완료\n 소요시간:', time_TLP_IDP_미등록)
print('------------------------------------------')     


# 1-11. Collection - TLP_IDP_T_earning
T_earning = pd.read_excel(file_name, sheet_name = sheet_name_TLP_IDP_T_earning)
T_earning = T_earning[T_earning['이수여부'] == 'Y']       # T_earning 은 이수여부 'Y' 만

time_T_earning = time.time() # timer setting

for _ in range(len(T_earning)):
  # var initializing
  사번=성명=최상위=실본부=팀=직책=메일=과정명=이수여부=채널=구분 = '' 
  학습시간 = 0.0
  
  # T_earning table에서 뽑을 수 있는 정보
  사번, 메일, 성명 = T_earning.iloc[_]['사번'], T_earning.iloc[_]['이메일'], T_earning.iloc[_]['성명']
  과정명, 이수여부 = T_earning.iloc[_]['과정명'], T_earning.iloc[_]['이수여부']
  학습시간 = format(float(T_earning.iloc[_]['학습시간(분)'])/60.0, ".3f")
  채널, 구분 = '러닝월드 T-earning', 'TLP'

  # 구성원정보와 join해서 얻는 정보
  특정_구성원정보 = 구성원정보[구성원정보['사번'] == 사번] # TLP는사번이 key
  try:
    최상위, 실본부, 팀, 직책 = 특정_구성원정보.iloc[0]['센터Lv'], 특정_구성원정보.iloc[0]['실/본부 Lv'], 특정_구성원정보.iloc[0]['팀 Lv'], 특정_구성원정보.iloc[0]['직책']
  except:
    최상위=실본부=팀=직책 = '구성원정보_미등록자'

  # output_종합 데이터 생성
  output_종합.loc[cnt] = [사번, 성명, 최상위, 실본부, 팀, 직책,
                        메일, 과정명, 학습시간, 이수여부, 채널, 구분]

  cnt += 1

  if cnt % 1000 == 0:
    print('T_earning', cnt, '번째 row 생성 완료 소요시간:', time.time() - time_T_earning)   

time_T_earning = time.time() - time_T_earning # time_T_earning processing time
print('[T_earning] 처리 완료\n 소요시간:', time_T_earning)
print('------------------------------------------')     


# 1-12. output_종합 tuning
# 직책 없는 사람들 nan -> '' 대체, 학습시간(시간) float으로 변환 
output_직책 = list(output_종합['직책'])
for _ in range(len(output_직책)):
  if str(output_직책[_]) == 'nan':
    output_직책[_] = ''
output_종합['직책'] = output_직책

output_시간 = list(output_종합['학습시간(시간)'])
for _ in range(len(output_시간)):
  output_시간[_] = float(output_시간[_]) 
output_종합['학습시간(시간)'] = output_시간


# 1-13. save output
time_total = time.time() - time_start
print('\n================================\n[종합] 테이블 생성 완료!\n 총 생성 데이터 수:', cnt, '개\n 총 소요시간:', time_total, '초')
output_종합.to_csv(output_종합_파일명,encoding='utf-8-sig', index=False) # utf-8-sig로 인코딩하면 엑셀로 열어도 한글 안깨짐


    

1000 번째 row 생성 완료 소요시간: 21.63322114944458
2000 번째 row 생성 완료 소요시간: 31.292236328125
3000 번째 row 생성 완료 소요시간: 40.904215574264526
4000 번째 row 생성 완료 소요시간: 50.85689163208008
5000 번째 row 생성 완료 소요시간: 61.521291971206665
6000 번째 row 생성 완료 소요시간: 72.2747950553894
7000 번째 row 생성 완료 소요시간: 83.45144724845886
8000 번째 row 생성 완료 소요시간: 94.38776850700378
9000 번째 row 생성 완료 소요시간: 105.80270266532898
10000 번째 row 생성 완료 소요시간: 117.87904238700867
11000 번째 row 생성 완료 소요시간: 130.29844975471497
12000 번째 row 생성 완료 소요시간: 142.8045835494995
13000 번째 row 생성 완료 소요시간: 156.1524908542633
14000 번째 row 생성 완료 소요시간: 169.86764311790466
15000 번째 row 생성 완료 소요시간: 183.85941219329834
16000 번째 row 생성 완료 소요시간: 197.8019518852234
17000 번째 row 생성 완료 소요시간: 212.4261543750763
18000 번째 row 생성 완료 소요시간: 226.70260453224182
19000 번째 row 생성 완료 소요시간: 241.5815577507019
20000 번째 row 생성 완료 소요시간: 257.2179808616638
21000 번째 row 생성 완료 소요시간: 272.59418201446533
22000 번째 row 생성 완료 소요시간: 288.77794766426086
23000 번째 row 생성 완료 소요시간: 305.44574069976807
24000 번째 row

# **PART 2. [학습시간]**

PART1 결과물인 output_종합.csv 파일이 있는지 확인하세요.


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)


import time
time_start = time.time() # timer setting

print('\n================================\n[학습시간] 처리 시작')

# 2-0. load tables
구성원정보 = pd.read_excel(file_name, sheet_name = sheet_name_구성원정보)
output_종합 = pd.read_csv(output_종합_파일명)
output_종합 = output_종합[output_종합['팀 Lv'] != '구성원정보_미등록자']   # 구성원정보 미등록자는 제외

학습시간목표 = 100

# 직책 없는 사람들 nan -> '' 변환 
output_직책 = list(output_종합['직책'])
for _ in range(len(output_직책)):
  if str(output_직책[_]) == 'nan':
    output_직책[_] = ''
output_종합['직책'] = output_직책

# studymate에서 수집된 애들은 이수여부 가 없어서 nan -> 일단 그대로?


# 2-1. 전사학습시간, 전사인원 연산
전사학습시간 =  sum(output_종합['학습시간(시간)'])

# 구성원 정보 테이블의 중복 제거 - 한 사람이 여러 아이디인 경우 제거
구성원정보_중복제거 = 구성원정보.drop_duplicates(['사번'], keep = 'first')
전사인원 = len(구성원정보_중복제거)


# 2-2. 각 센터별 학습시간, 인원 수 데이터 생성
학습시간_센터 = {}
센터_names = list(set(구성원정보['센터Lv']))

for 센터_name in 센터_names:
  # 센터별 학습시간 연산
  센터_학습시간 = output_종합[output_종합['최상위 Lv'] == 센터_name] # 특정 센터 필터링
  센터_name_학습시간 = sum(센터_학습시간['학습시간(시간)'])          # 센터별 학습시간 sum

  # 센터별 인원 수 연산
  센터_name_인원수 = len(구성원정보_중복제거[구성원정보_중복제거['센터Lv'] == 센터_name])

  학습시간_센터[센터_name] = {
      '학습시간' : 센터_name_학습시간,
      '인원수'   : 센터_name_인원수
  }


# 2-3. 실/본부별 학습시간, 인원 수 데이터 생성
#  '[실/본부직속]' 과 '[센터/단직속]' 은 제외하기 
학습시간_실본부 = {}
실본부_names = list(set(구성원정보['실/본부 Lv']))
if '[실/본부직속]' in 실본부_names:
  실본부_names.remove('[실/본부직속]')
if '[센터/단직속]' in 실본부_names:
  실본부_names.remove('[센터/단직속]')

for 실본부_name in 실본부_names:
  # 실/본부별 학습시간 연산
  실본부_학습시간 = output_종합[output_종합['실/본부 Lv'] == 실본부_name] # 특정 실/본부 필터링
  실본부_name_학습시간 = sum(실본부_학습시간['학습시간(시간)'])           # 학습시간 sum

  # 실/본부별 인원 수 연산
  실본부_name_인원수 = len(구성원정보_중복제거[구성원정보_중복제거['실/본부 Lv'] == 실본부_name])

  학습시간_실본부[실본부_name] = {
      '학습시간' : 실본부_name_학습시간,
      '인원수'   : 실본부_name_인원수
  }


# 2-4. 팀별 학습시간, 인원 수 데이터 생성
학습시간_팀 = {}
팀_names = list(set(구성원정보['팀 Lv']))

for 팀_name in 팀_names:
  # 팀별 학습시간 연산
  팀_학습시간 = output_종합[output_종합['팀 Lv'] == 팀_name] # 특정 팀 필터링
  팀_name_학습시간 = sum(팀_학습시간['학습시간(시간)'])      # 학습시간 연산

  # 팀별 인원 수 연산
  팀_name_인원수 = len(구성원정보_중복제거[구성원정보_중복제거['팀 Lv'] == 팀_name])

  학습시간_팀[팀_name] = {
      '학습시간' : 팀_name_학습시간,
      '인원수'   : 팀_name_인원수
  }

time_학습시간 = time.time() - time_start # 학습시간 processing time
print('[학습시간] 처리 완료\n 소요시간:', time_학습시간)



##################################################################
##################################################################

# searcher function 정의
def searcher():
  print('[검색] 사번을 아래에 입력하세요.')
  print('  사번_Output(실본부단위).xlsx, 사번_Output(팀단위).xlsx 파일이 생성됩니다.')
  print('  직속조직의 경우, Output(팀단위) 파일만 생성됩니다.')
  print('    입력 예시) 1112737')
  inp = input('입력: ')

  inp_사번 = int(inp)

  targets = 구성원정보_중복제거[구성원정보_중복제거['사번'] == inp_사번] # 사번 필터
  if len(targets) == 0: #
    print('잘못 입력하셨습니다. 사번을 확인하세요.')
  else:
    target_센터, target_실본부, target_팀 = list(targets['센터Lv'])[0], list(targets['실/본부 Lv'])[0], list(targets['팀 Lv'])[0]

  ##########################################
  # '[실/본부직속]', '[센터/단직속]'은 실본부가 없으므로 Output1(실본부단위) 생성 안 함
  if target_실본부 not in ['[실/본부직속]', '[센터/단직속]']:
    
    # 2-5. output1_대상 테이블 생성
    output1_대상 = pd.DataFrame(columns= ['',' '])
    output1_대상.loc[0] = ['사업부/센터', target_센터]
    output1_대상.loc[1] = ['실/본부', target_실본부]


    # 2-6. output1_평균학습시간 테이블 생성
    output1_평균학습시간 = pd.DataFrame(columns= ['구분','목표학습시간', '학습시간', '목표 달성률'])
    output1_평균학습시간.loc[0] = ['SK텔레콤 전사', 학습시간목표, format((전사학습시간/전사인원), ".3f"), (format(((전사학습시간/전사인원)/학습시간목표)*100, ".3f")+'%')]
    output1_평균학습시간.loc[1] = [target_센터, 학습시간목표, format((학습시간_센터[target_센터]['학습시간']/학습시간_센터[target_센터]['인원수']), ".3f"), (format(((학습시간_센터[target_센터]['학습시간']/학습시간_센터[target_센터]['인원수'])/학습시간목표)*100, ".3f")+'%')]
    output1_평균학습시간.loc[2] = [target_실본부, 학습시간목표, format((학습시간_실본부[target_실본부]['학습시간']/학습시간_실본부[target_실본부]['인원수']), ".3f"), (format(((학습시간_실본부[target_실본부]['학습시간']/학습시간_실본부[target_실본부]['인원수'])/학습시간목표)*100, ".3f")+'%')]


    # 2-7. output1_학습현황 테이블 생성
    output1_학습현황 = pd.DataFrame(columns= ['순번','실/본부', '팀', '평균학습시간', (target_실본부 + ' 평균대비')])

    특정_구성원정보 = 구성원정보[구성원정보['실/본부 Lv'] == target_실본부] # 실본부로 필터링
    팀_names = list(set(특정_구성원정보['팀 Lv']))                          # 해당 실본부에 속한 팀명 추출
    
    cnt_학습현황 = 0
    그룹평균 = 학습시간_실본부[target_실본부]['학습시간']/학습시간_실본부[target_실본부]['인원수']

    for 팀_name in 팀_names:
      팀평균 = 학습시간_팀[팀_name]['학습시간']/학습시간_팀[팀_name]['인원수']
      output1_학습현황.loc[cnt_학습현황] = [cnt_학습현황+1, target_실본부, 팀_name, format(팀평균, ".3f"), format(팀평균-그룹평균, ".3f")]
      cnt_학습현황 += 1


    # 2-8. output1 save file
    output1_filename = inp + '_Output(실본부단위).xlsx'
    writer = pd.ExcelWriter(output1_filename, engine='xlsxwriter')

    #시트별 데이터 추가
    output1_대상.to_excel(writer, sheet_name= '검색대상', index=False, header=False)
    output1_평균학습시간.to_excel(writer, sheet_name= '평균학습시간', index=False)
    output1_학습현황.to_excel(writer, sheet_name= '학습현황', index=False)

    #시트별 옵션 지정 - 가운데 정렬 및 열 너비
    workbook = writer.book
    cell_format = workbook.add_format()
    cell_format.set_align('center')

    worksheet = writer.sheets['검색대상']
    worksheet.freeze_panes(1, 0)
    worksheet.set_column('A:A', 15, cell_format)
    worksheet.set_column('B:B', 30, cell_format)

    worksheet = writer.sheets['평균학습시간']
    worksheet.freeze_panes(1, 0)
    worksheet.set_column('A:A', 30, cell_format)
    worksheet.set_column('B:B', 15, cell_format)
    worksheet.set_column('C:C', 15, cell_format)
    worksheet.set_column('D:D', 15, cell_format)

    worksheet = writer.sheets['학습현황']
    worksheet.freeze_panes(1, 0)
    worksheet.set_column('A:A', 5, cell_format)
    worksheet.set_column('B:B', 30, cell_format)
    worksheet.set_column('C:C', 30, cell_format)
    worksheet.set_column('D:D', 15, cell_format)
    worksheet.set_column('E:E', 30, cell_format)

    # 엑셀 파일 저장
    writer.save()
    print(output1_filename, '  파일 생성 완료.')


  # 2-9. output2_대상 테이블 생성
  output2_대상 = pd.DataFrame(columns= ['',' '])
  output2_대상.loc[0] = ['사업부/센터', target_센터]
  output2_대상.loc[1] = ['실/본부', target_실본부]
  output2_대상.loc[2] = ['팀', target_팀]
  output2_대상.loc[3] = ['팀인원', 학습시간_팀[target_팀]['인원수']]


  # 2-10. output2_평균학습시간 테이블 생성
  output2_평균학습시간 = pd.DataFrame(columns= ['구분','목표학습시간', '학습시간', '목표 달성률'])
  output2_평균학습시간.loc[0] = ['SK텔레콤 전사', 학습시간목표, format((전사학습시간/전사인원), ".3f"), (format(((전사학습시간/전사인원)/학습시간목표)*100, ".3f")+'%')]
  output2_평균학습시간.loc[1] = [target_센터, 학습시간목표, format((학습시간_센터[target_센터]['학습시간']/학습시간_센터[target_센터]['인원수']), ".3f"), (format(((학습시간_센터[target_센터]['학습시간']/학습시간_센터[target_센터]['인원수'])/학습시간목표)*100, ".3f")+'%')]
  output2_평균학습시간.loc[2] = [target_실본부, 학습시간목표, format((학습시간_실본부[target_실본부]['학습시간']/학습시간_실본부[target_실본부]['인원수']), ".3f"), (format(((학습시간_실본부[target_실본부]['학습시간']/학습시간_실본부[target_실본부]['인원수'])/학습시간목표)*100, ".3f")+'%')]
  output2_평균학습시간.loc[3] = [target_팀, 학습시간목표, format((학습시간_팀[target_팀]['학습시간']/학습시간_팀[target_팀]['인원수']), ".3f"), (format(((학습시간_팀[target_팀]['학습시간']/학습시간_팀[target_팀]['인원수'])/학습시간목표)*100, ".3f")+'%')]


  # 2-11. output2_학습현황 테이블 생성
  output2_학습현황 = pd.DataFrame(columns= ['순번','사번', '성명', '총 학습시간', 'mySUNI 학습시간', 'TLP 학습시간'])

  특정_구성원정보 = 구성원정보_중복제거[구성원정보_중복제거['팀 Lv'] == target_팀]      # 팀으로 필터링
  특정_구성원정보 = 특정_구성원정보.sort_values('사번') # 사번순 정렬

  for _ in range(len(특정_구성원정보)):
    순번, 사번, 성명 = _+1, 특정_구성원정보.iloc[_]['사번'], 특정_구성원정보.iloc[_]['성명']
    총학습, mySUNI, TLP = 0,0,0
    
    output_특정인 = output_종합[output_종합['사번'] == str(특정_구성원정보.iloc[_]['사번'])]      # 사번으로 output_종합 필터링
    if len(output_특정인) != 0:
      총학습 = sum(output_특정인['학습시간(시간)'])

      output_특정인_mySUNI = output_특정인[output_특정인['구분'] == 'mySUNI']
      if len(output_특정인_mySUNI) !=0:
        mySUNI = sum(output_특정인_mySUNI['학습시간(시간)'])

      output_특정인_TLP = output_특정인[output_특정인['구분'] == 'TLP']
      if len(output_특정인_TLP) !=0:
        TLP = sum(output_특정인_TLP['학습시간(시간)'])

    output2_학습현황.loc[_] = [순번, 사번, 성명, format(총학습, ".3f"), format(mySUNI, ".3f"), format(TLP, ".3f")]


  # 2-12. output2_학습이력 테이블 생성
  output2_학습이력 = pd.DataFrame(columns= ['순번','사번', '이름', '과정명', '학습시간', '학습구분'])

  output_이력 = output_종합[output_종합['팀 Lv'] == target_팀] 
  output_이력 = output_이력.sort_values('사번') # 사번순 정렬

  for _ in range(len(output_이력)):
    output2_학습이력.loc[_] = [_+1, output_이력.iloc[_]['사번'], output_이력.iloc[_]['성명'], output_이력.iloc[_]['과정명'], output_이력.iloc[_]['학습시간(시간)'], output_이력.iloc[_]['구분']]


  # 2-13. output2 save file
  output2_filename = inp + '_Output(팀단위).xlsx'
  writer = pd.ExcelWriter(output2_filename, engine='xlsxwriter')

  #시트별 데이터 추가
  output2_대상.to_excel(writer, sheet_name= '검색대상', index=False, header=False)
  output2_평균학습시간.to_excel(writer, sheet_name= '평균학습시간', index=False)
  output2_학습현황.to_excel(writer, sheet_name= '학습현황', index=False)
  output2_학습이력.to_excel(writer, sheet_name= '학습이력', index=False)

  #시트별 옵션 지정 - 가운데 정렬 및 열 너비
  workbook = writer.book
  cell_format = workbook.add_format()
  cell_format.set_align('center')

  worksheet = writer.sheets['검색대상']
  worksheet.freeze_panes(1, 0)
  worksheet.set_column('A:A', 15, cell_format)
  worksheet.set_column('B:B', 30, cell_format)

  worksheet = writer.sheets['평균학습시간']
  worksheet.freeze_panes(1, 0)
  worksheet.set_column('A:A', 30, cell_format)
  worksheet.set_column('B:B', 15, cell_format)
  worksheet.set_column('C:C', 15, cell_format)
  worksheet.set_column('D:D', 15, cell_format)

  worksheet = writer.sheets['학습현황']
  worksheet.freeze_panes(1, 0)
  worksheet.set_column('A:A', 5, cell_format)
  worksheet.set_column('B:B', 10, cell_format)
  worksheet.set_column('C:C', 15, cell_format)
  worksheet.set_column('D:D', 15, cell_format)
  worksheet.set_column('E:E', 15, cell_format)
  worksheet.set_column('F:F', 15, cell_format)

  worksheet = writer.sheets['학습이력']
  worksheet.freeze_panes(1, 0)
  worksheet.set_column('A:A', 5, cell_format)
  worksheet.set_column('B:B', 10, cell_format)
  worksheet.set_column('C:C', 15, cell_format)
  worksheet.set_column('D:D', 50)
  worksheet.set_column('E:E', 15, cell_format)
  worksheet.set_column('F:F', 15, cell_format)

  # 엑셀 파일 저장
  writer.save()
  print(output2_filename, ' 파일 생성 완료.')
   



[학습시간] 처리 시작
[학습시간] 처리 완료
 소요시간: 12.766980171203613


# **검색기능 사용시 아래 셀을 실행해 주세요.**

In [ ]:
searcher()

[검색] 사번을 아래에 입력하세요.
  사번_Output(실본부단위).xlsx, 사번_Output(팀단위).xlsx 파일이 생성됩니다.
  직속조직의 경우, Output(팀단위) 파일만 생성됩니다.
    입력 예시) 1112737
입력: 1109839
1109839_Output(실본부단위).xlsx   파일 생성 완료.
1109839_Output(팀단위).xlsx  파일 생성 완료.
